# Baseline 1

In [1]:
!git clone https://github.com/Shaun-le/VNABSA.git

Cloning into 'VNABSA'...
remote: Enumerating objects: 695, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 695 (delta 49), reused 105 (delta 40), pack-reused 572
Receiving objects: 100% (695/695), 12.84 MiB | 18.84 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [2]:
cd VNABSA/InstABSA

/kaggle/working/VNABSA/InstABSA


In [3]:
%%capture
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install sentencepiece

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [5]:
import os

task_name = 'joint_task'
experiment_name = 'mydata'
model_checkpoint = 'VietAI/vit5-base'
print('Experiment Name: ', experiment_name)
model_out_path = '/kaggle/working/Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  mydata
Model output path:  /kaggle/working/Models/joint_task/VietAIvit5-base-mydata


In [6]:
# Load the data
id_train_file_path = '/kaggle/working/VNABSA/InstABSA/Dataset/mydata/mydata_train.csv'
id_test_file_path = '/kaggle/working/VNABSA/InstABSA/Dataset/mydata/mydata_test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [7]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    
}

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.675700
1000,0.160700
1500,0.097000
2000,0.055900
2500,0.033800
3000,0.022600


In [14]:
# Load the data
id_train_file_path = '/kaggle/working/VNABSA/InstABSA/Dataset/mydata/mydata_train.csv'
id_test_file_path = '/kaggle/working/VNABSA/InstABSA/Dataset/mydata/mydata_test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [ ]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Model loaded to:  cuda


100%|██████████| 398/398 [09:26<00:00,  1.42s/it]


Model loaded to:  cuda


 98%|█████████▊| 98/100 [02:22<00:02,  1.43s/it]

In [ ]:
model_trainer.save_model('/kaggle/working/checkpoints')

In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/checkpoints")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/checkpoints")

bos_instruction = """Định nghĩa: Kết quả đầu ra sẽ bao gồm các khía cạnh (cả tiềm ẩn và rõ ràng) và hướng cảm xúc của các khía cạnh. Trong trường hợp không có bất kỳ khía cạnh nào, kết quả đầu ra sẽ là noaspectterm:none.
        Positive example 1-
        input: giao hàng nhanh. nhận đc áo đẹp hơn cả mong đợi! vải áo và đường may rất đẹp đóng gói rất xịn xò, đánh giá 5 sao lần sau sẽ mua ủng hộ shop tiếp ạ.
        output: giao hàng nhanh:positive, áo đẹp hơn cả mong đợi:positive
        Positive example 2-
        input: thấy giá rẻ, khá là đắn đo phân vân với sản phẩm khác. lựa chọn tới lui cuối cùng mua thử. thật đáng đồng tiền, vải mỏng mịn, k dày thô như mấy thun cá sấu khác, chất mát vô cùng. fom đẹp, size vừa vặn. chồng mình thử tới đâu khen tới đó. shop rất nhiệt tình, giao hàng siêu nhanh.
        output: thật đáng đồng tiền:positive, vải mỏng mịn:positive, fom đẹp, size vừa vặn:positive, shop rất nhiệt tình, giao hàng siêu nhanh:positive
        Negative example 1-
        input: chất liệu:vải mỏng màu sắc:trắng đúng không đúnh với mô tả sai kích cỡ.
        output: vải mỏng:negative, không đúnh với mô tả sai kích cỡ:negative
        Negative example 2-
        input: k to như mình nghĩ.
        output: k to như mình nghĩ:negative
        Neutral example 1-
        input: chất liệu bình thường đúng với mô tả:bình thường màu sắc bình thường hình ảnh chỉ mang tính chất nhận xu.
        output: chất liệu bình thường:neutral, màu sắc bình thường:neutral
        Neutral example 2-
        input: đúng với mô tả:ýe chất liệu bthg màu sắc:den.
        output: chất liệu bthg:neutral
        Bây giờ hoàn thành example sau-
        input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = 'màu sắc:đúng chuẩn màu đúng với mô tả:oki chất liệu:vải mát shop giao hàng nhanh, gói hàng siêu chất lượng và cũng dth lắm ạ. ủng hộ shop.'

tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

Model output:  'term': 'chất liệu:vải mát', 'polar


# Baseline 2

In [1]:
%%capture
!conda create -n newCondaEnvironment -c cctbx202208 python=3.8 -y
!source /opt/conda/bin/activate newCondaEnvironment && conda install -c cctbx202208 python -y
!/opt/conda/envs/newCondaEnvironment/bin/python3 --version
!echo 'print("Hello, World!")' > test.py
!/opt/conda/envs/newCondaEnvironment/bin/python3 test.py
!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3
!sudo rm /opt/conda/bin/python3.10
!sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3.10
!sudo rm /opt/conda/bin/python
!sudo ln -s /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python

In [2]:
!python --version

Python 3.8.13


In [3]:
!git clone https://github.com/Shaun-le/VNABSA.git

Cloning into 'VNABSA'...
remote: Enumerating objects: 695, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 695 (delta 49), reused 105 (delta 40), pack-reused 572
Receiving objects: 100% (695/695), 12.84 MiB | 20.20 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [4]:
cd VNABSA/Multi-view/

/kaggle/working/VNABSA/Multi-view


In [5]:
%%capture
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install -r requirements.txt

In [6]:
%%writefile /kaggle/working/VNABSA/Multi-view/scripts/run_main.sh
set -ex

export CUDA_VISIBLE_DEVICES=0

declare -A TASK_DATA
#TASK_DATA[asqp]="rest15 rest16"
#TASK_DATA[acos]="laptop16 rest16"
#TASK_DATA[aste]="laptop14"
TASK_DATA[tasd]="rest15"

cd src

# for SVP_TYPE in heuristic rand rank 
for TASK in tasd
do
for DATA in ${TASK_DATA[${TASK}]}
do
for DATA_RATIO in 1.0
do
for SEED in 5
do
for K in 5
# for K in 3 7 15
do
INFER_PATH=$K
CTRL_TOKEN=post
OUT_DIR="../outputs/$TASK/${DATA}/top${K}_${CTRL_TOKEN}_data${DATA_RATIO}"

mkdir -p $OUT_DIR

python main.py \
    --data_path "../data/" \
    --dataset $DATA \
    --model_name_or_path t5-base \
    --output_dir $OUT_DIR \
    --num_train_epochs 1 \
    --save_top_k 0 \
    --task $TASK \
    --top_k $K \
    --ctrl_token $CTRL_TOKEN \
    --multi_path \
    --num_path $INFER_PATH \
    --seed $SEED \
    --train_batch_size 16 \
    --gradient_accumulation_steps 1 \
    --learning_rate 1e-4 \
    --lowercase \
    --sort_label \
    --data_ratio $DATA_RATIO \
    --check_val_every_n_epoch 10  \
    --agg_strategy vote \
    --eval_batch_size 64 \
    --constrained_decode \
    --do_train \
    | tee ${OUT_DIR}/train.log \
    2> ${OUT_DIR}/train.err
    # --model_name_or_path "PATH TO THE CHECKPOINT" \ # configure the checkpoint path to eval

    # --load_path_cache \
    # --single_view_type $SVP_TYPE \
    # --load_ckpt_name "ckpt path" \
    # > $OUT_DIR/train.log 2>&1&
done
done
done
done
done
# done

Overwriting /kaggle/working/VNABSA/Multi-view/scripts/run_main.sh


In [7]:
!bash scripts/run_main.sh

+ export CUDA_VISIBLE_DEVICES=0
+ CUDA_VISIBLE_DEVICES=0
+ declare -A TASK_DATA
+ TASK_DATA[tasd]=rest15
+ cd src
+ for TASK in tasd
+ for DATA in ${TASK_DATA[${TASK}]}
+ for DATA_RATIO in 1.0
+ for SEED in 5
+ for K in 5
+ INFER_PATH=5
+ CTRL_TOKEN=post
+ OUT_DIR=../outputs/tasd/rest15/top5_post_data1.0
+ mkdir -p ../outputs/tasd/rest15/top5_post_data1.0
+ python main.py --data_path ../data/ --dataset rest15 --model_name_or_path t5-base --output_dir ../outputs/tasd/rest15/top5_post_data1.0 --num_train_epochs 1 --save_top_k 0 --task tasd --top_k 5 --ctrl_token post --multi_path --num_path 5 --seed 5 --train_batch_size 16 --gradient_accumulation_steps 1 --learning_rate 1e-4 --lowercase --sort_label --data_ratio 1.0 --check_val_every_n_epoch 10 --agg_strategy vote --eval_batch_size 64 --constrained_decode --do_train
+ tee ../outputs/tasd/rest15/top5_post_data1.0/train.log
Random seed set as 5

 ============================== NEW EXP: tasd on rest15 ============================== 

Downlo

# Baseline3

In [1]:
!git clone https://github.com/Shaun-le/VNABSA.git

Cloning into 'VNABSA'...
remote: Enumerating objects: 952, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 952 (delta 16), reused 32 (delta 9), pack-reused 901
Receiving objects: 100% (952/952), 14.88 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (469/469), done.


In [2]:
cd VNABSA/BERT

/kaggle/working/VNABSA/BERT


In [3]:
%%capture
!pip install transformers['torch']

In [4]:
#@title Choose a dataset and a task { run: "auto", display-mode: "form" }
base_dir = "/kaggle/working/VNABSA/BERT" #@param {type:"string"}
dataset_type = "mydata" #@param ["sentihood", "semeval2014"]
task = "single" #@param ["single"]

In [5]:
import pandas as pd
import random


if dataset_type == "sentihood":
    id2label = {0: "None", 1: "Positive", 2: "Negative"}
    label2id = {"None": 0, "Positive": 1, "Negative": 2}
elif dataset_type == "semeval2014":
    id2label = {0: "positive", 1: "neutral", 2: "negative", 3: "conflict", 4: "none"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2, "conflict": 3, "none": 4}
elif dataset_type == 'mydata':
    id2label = {0: "positive", 1: "neutral", 2: "negative"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2}

if dataset_type == "sentihood":
    num_classes = 3
    locations = ["location_1_", "location_2_"]
    aspects = ["general", "price", "safety", "transit location"]
elif dataset_type == "semeval2014":
    num_classes = 5
    locations = [""]
    aspects = ["ambience", "anecdotes", "food", "price", "service"]
elif dataset_type == 'mydata':
    num_classes = 3
    locations = [""]
    aspects = ["design", "general", "material", "price", "service"]

def get_dataset(path):
    original_sentences = []
    labels = []
    data = pd.read_csv(path, sep="\t").values.tolist()
    for row in data:
        original_sentences.append(row[1])
        labels.append(row[3])
    return original_sentences, labels


train_original_sentences = {}
train_labels = {}
val_original_sentences = {}
val_labels = {}
test_original_sentences = {}
test_labels = {}

for location in locations:
    train_original_sentences[location] = {}
    train_labels[location] = {}
    val_original_sentences[location] = {}
    val_labels[location] = {}
    test_original_sentences[location] = {}
    test_labels[location] = {}
    for aspect in aspects:
        train_original_sentences[location][aspect], train_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/train.csv")
        if dataset_type == "sentihood":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/dev.csv")
        elif dataset_type == "semeval2014":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")
        elif dataset_type == 'mydata':
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")
        test_original_sentences[location][aspect], test_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")

In [6]:
from transformers import BertTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

train_encodings = {}
val_encodings = {}
test_encodings = {}
for location in locations:
    train_encodings[location] = {}
    val_encodings[location] = {}
    test_encodings[location] = {}
    for aspect in aspects:
        train_encodings[location][aspect] = tokenizer(train_original_sentences[location][aspect], truncation=True, padding=True)
        val_encodings[location][aspect] = tokenizer(val_original_sentences[location][aspect], truncation=True, padding=True)
        test_encodings[location][aspect] = tokenizer(test_original_sentences[location][aspect], truncation=True, padding=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = {}
val_dataset = {}
test_dataset = {}
for location in locations:
    train_dataset[location] = {}
    val_dataset[location] = {}
    test_dataset[location] = {}
    for aspect in aspects:
        train_dataset[location][aspect] = ABSA_Dataset(train_encodings[location][aspect], train_labels[location][aspect])
        val_dataset[location][aspect] = ABSA_Dataset(val_encodings[location][aspect], val_labels[location][aspect])
        test_dataset[location][aspect] = ABSA_Dataset(test_encodings[location][aspect], test_labels[location][aspect])

In [8]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig, AutoModelForSequenceClassification, AutoConfig
from transformers import logging
import gc
import pandas as pd
import numpy as np
from scipy.special import softmax


logging.set_verbosity_debug()


epochs = 1
batch_size = 24

header = ["predicted_label"]
for label in label2id.keys():
    header.append(label)

config = AutoConfig.from_pretrained(
        'vinai/phobert-base',
        architectures = ['AutoModelForSequenceClassification'],
        hidden_size = 768,
        num_hidden_layers = 12,
        num_attention_heads = 12,
        hidden_dropout_prob = 0.1,
        num_labels = num_classes
    )

for location in locations:
    for aspect in aspects:
        num_steps = len(train_dataset[location][aspect]) * epochs // batch_size
        warmup_steps = num_steps // 10  # 10% of the training steps
        #save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


        training_args = TrainingArguments(
            output_dir = f'{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/',
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            warmup_steps = warmup_steps,
            weight_decay = 0.01,
            logging_dir = f'{base_dir}/logs/{dataset_type}/BERT-single/{location}{aspect}/',
            logging_steps = 10,
            evaluation_strategy = 'epoch',
            learning_rate = 2e-5,
            #save_steps = save_steps,
            seed=21
        )

        model = AutoModelForSequenceClassification.from_pretrained('vinai/phobert-base', config=config)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset[location][aspect],
            eval_dataset=val_dataset[location][aspect]
        )

        trainer.train()

        #model.save_pretrained(f"{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/last_step")

        results = trainer.predict(test_dataset[location][aspect])

        scores = [softmax(prediction) for prediction in results.predictions]
        predicted_labels = [np.argmax(x) for x in scores]

        csv_output = np.insert(scores, 0, predicted_labels, axis=1)
        df = pd.DataFrame(csv_output)
        df[0] = df[0].astype("int")
        df.to_csv(f"{base_dir}/results/{dataset_type}/BERT-single/{location}{aspect}.csv", index=False, header=header)

        del training_args
        del model
        del trainer
        del results
        del scores
        del predicted_labels
        del csv_output
        del df
        gc.collect()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/a47f76f1ac0b6b89f82d15dbe9829935a0014c38/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "AutoModelForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "lay

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/a47f76f1ac0b6b89f82d15dbe9829935a0014c38/pytorch_model.bin
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.552200,0.589321


...

Epoch,Training Loss,Validation Loss
1,0.495400,0.398532


***** Running Evaluation *****
  Num examples = 392
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 392
  Batch size = 24


Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/a47f76f1ac0b6b89f82d15dbe9829935a0014c38/pytorch_model.bin
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.d

Epoch,Training Loss,Validation Loss
1,0.290100,0.280169


***** Running Evaluation *****
  Num examples = 634
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 634
  Batch size = 24


In [9]:
scores, predicted_labels, true_labels = [],[],[]
semeval_aspects = ["design", "general", "material", "price", "service"]
data = {}
for aspect in semeval_aspects:
    data[aspect] = pd.read_csv(f"/kaggle/working/VNABSA/BERT/results/mydata/BERT-single/{aspect}.csv").values.tolist()
for i in range(len(semeval_aspects)):
    for j in range(len(data[semeval_aspects[i]])):
        scores.append(data[semeval_aspects[i]][j][1:])
        predicted_labels.append(int(data[semeval_aspects[i]][j][0]))
        
data = {}
for aspect in semeval_aspects:
    data[aspect] = pd.read_csv(f"/kaggle/working/VNABSA/BERT/data/mydata/BERT-single/{aspect}/test.csv", sep='\t').values.tolist()
for i in range(len(semeval_aspects)):
    for j in range(len(data[semeval_aspects[i]])):
        true_labels.append(data[semeval_aspects[i]][j][3])

In [10]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

print(f1_score(true_labels, predicted_labels, average='micro'))
print(recall_score(true_labels, predicted_labels, average='micro'))
print(precision_score(true_labels, predicted_labels, average='micro'))

0.8636231462634487
0.8636231462634487
0.8636231462634487


In [11]:
'''import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import evaluation
evaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")'''

'import sys\nif base_dir not in sys.path:\n    sys.path.insert(0, f\'{base_dir}/\')\nimport evaluation\nevaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")'

# Baseline4

In [1]:
!python --version

Python 3.10.12


In [1]:
!git clone https://github.com/Shaun-le/VNABSA.git

Cloning into 'VNABSA'...
remote: Enumerating objects: 952, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 952 (delta 16), reused 32 (delta 9), pack-reused 901
Receiving objects: 100% (952/952), 14.88 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (469/469), done.


In [2]:
cd VNABSA/Gas

/kaggle/working/VNABSA/Gas


In [3]:
#%%capture
!pip install sentencepiece editdistance transformers pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 13.9 MB/s eta 0:00:00


In [4]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 532.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 70.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1
    Uninstalling torchaudio-2.0.1:
      Successfully uninstalled torchaudio-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [5]:
%%writefile /kaggle/working/VNABSA/Gas/run.sh
#!/usr/bin/env bash

python main.py --task tasd \
            --dataset mydata \
            --model_name_or_path VietAI/vit5-base \
            --paradigm annotation \
            --n_gpu 0 \
            --do_train \
            --do_direct_eval \
            --train_batch_size 1 \
            --gradient_accumulation_steps 2 \
            --eval_batch_size 16 \
            --learning_rate 3e-4 \
            --num_train_epochs 10

Overwriting /kaggle/working/VNABSA/Gas/run.sh


In [6]:
!bash run.sh

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

 ============================== NEW EXP: TASD on mydata ============================== 

Here is an example (from dev set) under `annotation` paradigm:
Total examples = 700
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Input : đúng với mô tả:ok oke cx đc màu chữ nhạt